## general_import.py transposed to a .ipynb file for debugging

In [1]:
import numpy as np
import pandas as pd
import os
from DB2SalesforceAPI import DB2SalesforceAPI

sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"sf2021shitOPmlIiFMLnrudgC6oSX0WV1T", 
}

In [2]:
cwd = os.getcwd()

In [3]:
#input new filename here
filename = 'Long-term Users_not recently active.csv'
#'NCN URE UCEE SURF SURI Contact List.xlsx' #'Citrine Webinar Sent.csv'
f_type = filename.split('.')[-1]

In [4]:
#pandas import
if f_type == 'csv':
    idf = pd.read_csv(cwd+'/custom objects/'+filename)#,encoding='utf-16',sep='\t')
                      #,encoding="cp1252",error_bad_lines=False)#,usecols=[12])#col_fields)
elif f_type == 'xlsx' or f_type == 'xls':
    xl = pd.ExcelFile(cwd+'/custom objects/'+filename)
    print(xl.sheet_names)# see all sheet names
    sheet_names = xl.parse(xl.sheet_names) #this already performs an import
    
    idf = pd.read_excel(cwd+'/Data Files/'+filename,sheet_name=0,header=0)

print(idf.head(2))

  First Name  Last Name                 Email  nanoHUB user ID  \
0      hasna   kudilatt       hk462@cam.ac.uk           195805   
1    Marcelo  Pavanello  pavanello@fei.edu.br           201273   

        Contact ID                         new_preferences_survey_url  \
0  0035w000034Jz2P  https://nanohub.org/reply/email-subscriptions?...   
1  0035w000034K3AJ  https://nanohub.org/reply/email-subscriptions?...   

                                  nh_use_survey_link  updatesnews  \
0  https://nanohub.org/reply/pages/2?code=ZgOFtCF...            0   
1  https://nanohub.org/reply/pages/2?code=SkyzXu5...            0   

   personalizedcommunication  
0                          1  
1                          1  


In [5]:
idf = idf[['nanoHUB user ID','Email','Contact ID']]

In [6]:
# idf = idf.dropna(subset=['Email Address'])
display(idf)

,nanoHUB user ID,Email,Contact ID
0,195805,hk462@cam.ac.uk,0035w000034Jz2P
1,201273,pavanello@fei.edu.br,0035w000034K3AJ
2,201547,ahaldar@bu.edu,0035w000034K3Mz
3,201817,284461042@qq.com,0035w000034K3ZL
4,209664,reshmaparthasarathy06@gmail.com,0035w000034K2DT
...,...,...,...
11647,116789,i.zadorozhnyi@fz-juelich.de,0035w000034JrT0
11648,117582,nirmalharsh53@gmail.com,0035w000034JsIZ
11649,117584,greenlight2005@gmail.com,0035w000034JsIb
11650,117588,plapas@physics.utexas.edu,0035w000034JsIf


In [7]:
# if engagement venue does not exist, then create a flag with that entry
try:
    #if engagement venue is specified
    print(idf['Engagement Venue'])
    print(idf['First Name'])
    idf = idf.rename(columns={'Engagement Venue':'Venue__c'})
    idf = idf.rename(columns={'First Name':'firstname','Last Name':'lastname'})
except:
    #decide the event
    #event_extract = xl.sheet_names[0]
    #idf['Engagement Venue'] = event_extract
#     idf['Venue__c'] = '2021 nano-challenge'
#     idf['MaRDA_Survey_Link__c'] = 'MaRDA_Survey_Link__c'
    idf['Venue__c'] = 'Long-term Users_not recently active' #'MaRDA 2021 Survey'
    
    try:
        #name extract
        names = idf['Name'].to_list()
        from copy import deepcopy
        fname = deepcopy(names)
        lname = deepcopy(names)
        for ind, val in enumerate(names):
            val = val.split(' ')
            fname[ind] = val[-1]
            lname[ind] = val[0]

        idf['firstname'] = fname
        idf['lastname'] = lname
        name_flag = True
    except:
        name_flag = False
        print('no name')

no name


In [8]:
#rename columns
idf = idf.rename(columns={'email':'Email','EMAIL':'Email','E-mail Address':'Email','Email Address':'Email'})
idf = idf.rename(columns={'Engagement Venue':'Venue__c'})
idf = idf.rename(columns={'First Name':'firstname','Last Name':'lastname'})
idf = idf.rename(columns={0:'Email'})
if name_flag == True:
    idf = idf.drop(columns='Name')#['NAME','LAST NAME','FIRST NAME'])

In [9]:
display(idf.head(5))

,nanoHUB user ID,Email,Contact ID,Venue__c
0,195805,hk462@cam.ac.uk,0035w000034Jz2P,Long-term Users_not recently active
1,201273,pavanello@fei.edu.br,0035w000034K3AJ,Long-term Users_not recently active
2,201547,ahaldar@bu.edu,0035w000034K3Mz,Long-term Users_not recently active
3,201817,284461042@qq.com,0035w000034K3ZL,Long-term Users_not recently active
4,209664,reshmaparthasarathy06@gmail.com,0035w000034K2DT,Long-term Users_not recently active


In [10]:
idf = idf.rename(columns={'Email ':'Email','nanoHUB user ID':'nanoHUB_user_ID__c'})

In [11]:
# Email and alternate email cross ref
primary_email = []
alt_email = []

for i,j in enumerate(idf['Email']):
    t_email = idf['Alternate email'].tolist()[i]
    if type(j) != str: #then primary email should take the entry from alt_emails
        primary_email.append(t_email)
        alt_email.append(None)
    else:
        primary_email.append(j)
        alt_email.append(t_email)

KeyError: 'Alternate email'

In [12]:
idf['Email'] = primary_email
idf['Alternate email'] = alt_email

ValueError: Length of values does not match length of index

In [13]:
idf.tail(5)

,nanoHUB_user_ID__c,Email,Contact ID,Venue__c
11647,116789,i.zadorozhnyi@fz-juelich.de,0035w000034JrT0,Long-term Users_not recently active
11648,117582,nirmalharsh53@gmail.com,0035w000034JsIZ,Long-term Users_not recently active
11649,117584,greenlight2005@gmail.com,0035w000034JsIb,Long-term Users_not recently active
11650,117588,plapas@physics.utexas.edu,0035w000034JsIf,Long-term Users_not recently active
11651,117599,bmcintyre4707@att.net,0035w000034JsIp,Long-term Users_not recently active


In [14]:
print(idf.shape)

(11652, 4)


In [15]:
idf = idf.dropna(subset=['Email'])
print(idf.shape)

(11652, 4)


In [16]:
#email check rows
grows = []
brows = []
for ind,val in enumerate(idf['Email'].to_list()):
    if '@' in val:
        grows.append(ind)
    else:
        brows.append(ind)    
idf = idf.iloc[grows,:].reset_index().iloc[:,1:]
print(idf.head(2))

   nanoHUB_user_ID__c                 Email       Contact ID  \
0              195805       hk462@cam.ac.uk  0035w000034Jz2P   
1              201273  pavanello@fei.edu.br  0035w000034K3AJ   

                              Venue__c  
0  Long-term Users_not recently active  
1  Long-term Users_not recently active  


In [17]:
print(len(grows))
print(len(brows))

11652
0


## Special case where we sent the emails from SF

In [21]:
idf = idf.rename(columns={'nanoHUB user ID': 'nanoHUB_user_ID__c'})

In [22]:
if 'nanoHUB_user_ID__c' in idf.columns:
    nh_ID_flag = True
    try:
        idf = idf.drop(columns=['new_preferences_survey_url','nh_use_survey_link','personalizedcommunication'])
        #idf['Venue__c'] = 'Citrine Material Science Workshop - S'
    except:
        #idf['Venue__c'] = 'Citrine Material Science Workshop - S'
    display(idf.head(5))

IndentationError: expected an indented block (<ipython-input-22-bc4a50e2247d>, line 8)

### Upsert Contacts with new Venue

In [18]:
import os
os_name = os.name
import platform
sys_name = platform.system() #Linux, Darwin, Windows

In [19]:
sys_name = 'Linux'

In [20]:
# salesforce queries for contact data
db_s = DB2SalesforceAPI(sf_login_params)

# deciding the queries
from copy import deepcopy
import_df_cols = deepcopy(idf.columns)
nh_id_flag = False
email_flag = False
if 'nanoHUB_user_ID__c' in import_df_cols:
    nh_id_flag = True

if 'Email' in import_df_cols:
    email_flag = True

Obtained Salesforce access token ...... True


In [21]:
if nh_id_flag == True and email_flag == True:
    sf_df = db_s.query_data('SELECT nanoHUB_user_ID__c, Email, Venue__c FROM Contact',sys_name=sys_name)
elif email_flag == True:
    sf_df = db_s.query_data('SELECT nanoHUB_user_ID__c, Email, Venue__c FROM Contact',sys_name=sys_name)

[Success] Bulk job creation successful. Job ID = 7505w00000VKWTdAAP
{"id":"7505w00000VKWTdAAP","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-04-16T15:58:22.000+0000","systemModstamp":"2021-04-16T15:58:22.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000VKWTdAAP","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-04-16T15:58:22.000+0000","systemModstamp":"2021-04-16T15:58:22.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000VKWTdAAP","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDat

In [22]:
# find all existing contacts
sf_emails = sf_df['Email'].to_list()
grows = []
brows = [] #dont need the sf_bad_rows as send to leads
sf_grows = []
for ind,val in enumerate(idf['Email'].to_list()):
    if val in sf_emails:
        grows.append(ind)
        sf_grows.append(sf_emails.index(val))
    else:
        brows.append(ind)

In [23]:
print(len(grows))
print(len(sf_grows))

11652
11652


In [29]:
type(idf['Alternate email'].to_list()[0])

KeyError: 'Alternate email'

In [30]:
# filter for alternate emails
for ind,val in enumerate(idf['Alternate email'].to_list()):
    if val in sf_emails and type(val) != float and type(val) != type(None):
        grows.append(ind)
        sf_grows.append(sf_emails.index(val))

KeyError: 'Alternate email'

In [22]:
# filter for duplicate row indexes
grows = list(set(grows))
brows = list(set(brows))
sf_grows = list(set(sf_grows))
print(len(grows))
print(len(brows))
print(len(sf_grows))

4140
0
4137


In [24]:
sf_df_match = sf_df.iloc[sf_grows,:].reset_index().iloc[:,1:]
idf_match = idf.iloc[grows,:].reset_index().iloc[:,1:]


In [25]:
display(sf_df_match.iloc[130:150,:])
display(idf_match.iloc[130:150,:])

,Email,Venue__c,nanoHUB_user_ID__c
130,jason.kilpatrick@ucd.ie,NaN,43977.0
131,mariesto@gmail.com,NaN,44410.0
132,ravmun@gmail.com,NaN,44412.0
133,vitavita@bigmir.net,NaN,44421.0
134,joern.schwandt@desy.de,NaN,44440.0
135,sunyuyin.david@gmail.com,NaN,44443.0
136,arunvt085@gmail.com,NaN,44872.0
137,tihomir.knezevic@fer.hr,NaN,44876.0
138,kgupta27@gmail.com,NaN,44890.0
139,sampo.laakso@vtt.fi,NaN,44903.0


,nanoHUB_user_ID__c,Email,Contact ID,Venue__c
130,43977,jason.kilpatrick@ucd.ie,0035w000031VtC6,Long-term Users_not recently active
131,44410,mariesto@gmail.com,0035w000031VtcS,Long-term Users_not recently active
132,44412,ravmun@gmail.com,0035w000031VtcU,Long-term Users_not recently active
133,44421,vitavita@bigmir.net,0035w000031Vtcd,Long-term Users_not recently active
134,44440,joern.schwandt@desy.de,0035w000031Vtcw,Long-term Users_not recently active
135,44443,sunyuyin.david@gmail.com,0035w000031Vtcz,Long-term Users_not recently active
136,44872,arunvt085@gmail.com,0035w000031Vtwf,Long-term Users_not recently active
137,44876,tihomir.knezevic@fer.hr,0035w000031Vtwj,Long-term Users_not recently active
138,44890,kgupta27@gmail.com,0035w000031Vtww,Long-term Users_not recently active
139,44903,sampo.laakso@vtt.fi,0035w000031Vtx9,Long-term Users_not recently active


In [26]:
# join the dataframes - then indicate hed__AlternateEmail__c
idf_match = pd.merge(sf_df_match,idf_match,left_on='Email',right_on='Email',how='outer')
display(idf_match)
print(idf_match.shape)

,Email,Venue__c_x,nanoHUB_user_ID__c_x,nanoHUB_user_ID__c_y,Contact ID,Venue__c_y
0,hk462@cam.ac.uk,NaN,195805.0,195805,0035w000034Jz2P,Long-term Users_not recently active
1,pavanello@fei.edu.br,NaN,201273.0,201273,0035w000034K3AJ,Long-term Users_not recently active
2,ahaldar@bu.edu,NaN,201547.0,201547,0035w000034K3Mz,Long-term Users_not recently active
3,284461042@qq.com,NaN,201817.0,201817,0035w000034K3ZL,Long-term Users_not recently active
4,reshmaparthasarathy06@gmail.com,NaN,209664.0,209664,0035w000034K2DT,Long-term Users_not recently active
...,...,...,...,...,...,...
11871,i.zadorozhnyi@fz-juelich.de,NaN,116789.0,116789,0035w000034JrT0,Long-term Users_not recently active
11872,nirmalharsh53@gmail.com,NaN,117582.0,117582,0035w000034JsIZ,Long-term Users_not recently active
11873,greenlight2005@gmail.com,NaN,117584.0,117584,0035w000034JsIb,Long-term Users_not recently active
11874,plapas@physics.utexas.edu,NaN,117588.0,117588,0035w000034JsIf,Long-term Users_not recently active


(11876, 6)


In [27]:
display(idf_match.iloc[:20,:])

,Email,Venue__c_x,nanoHUB_user_ID__c_x,nanoHUB_user_ID__c_y,Contact ID,Venue__c_y
0,hk462@cam.ac.uk,NaN,195805.0,195805,0035w000034Jz2P,Long-term Users_not recently active
1,pavanello@fei.edu.br,NaN,201273.0,201273,0035w000034K3AJ,Long-term Users_not recently active
2,ahaldar@bu.edu,NaN,201547.0,201547,0035w000034K3Mz,Long-term Users_not recently active
3,284461042@qq.com,NaN,201817.0,201817,0035w000034K3ZL,Long-term Users_not recently active
4,reshmaparthasarathy06@gmail.com,NaN,209664.0,209664,0035w000034K2DT,Long-term Users_not recently active
5,darien.perezmartin@gmail.com,NaN,201117.0,201117,0035w000034K2zB,Long-term Users_not recently active
6,wrheinhe@purdue.edu,NaN,201834.0,201834,0035w000034K3ZX,Long-term Users_not recently active
7,alexsu8592@gmail.com,NaN,202939.0,202939,0035w000034K4NC,Long-term Users_not recently active
8,nercisus@gmail.com,NaN,211614.0,211614,0035w000034K3ql,Long-term Users_not recently active
9,hotlolly12345@gmail.com,NaN,201143.0,201143,0035w000034K2zZ,Long-term Users_not recently active


In [28]:
## combine venues into a single col (include multiyear UREs)
## drop year
## drop alternate email
## drop duplicates

ven_x = idf_match['Venue__c_x'].to_list()
ven_y = idf_match['Venue__c_y'].to_list()
ven_joined = []

for i,j in enumerate(ven_x):
    temp = []
    if type(ven_y[i]) != float and type(ven_y[i]) != type(None):
        temp.append(ven_y[i])
    
    if type(j) != float and type(j) != type(None):
        temp.append(j)
    #print(temp)
    ven_joined.append(';'.join(temp))

In [29]:
idf_match.shape

(11876, 6)

In [30]:
len(ven_joined)

11876

In [31]:
## load ven_joined, check email for duplicates and join multiyear UREs
idf_match['ven_joined'] = ven_joined

unique_emails = idf_match['Email'].unique()
tt_venue_emails = {}
for i,j in enumerate(unique_emails):
    # determine all indexes with said unique_email
    temp2 = np.where(idf_match['Email'] == j)
    tt_venue_emails[j] = temp2

In [32]:
idf_match.head(5)

,Email,Venue__c_x,nanoHUB_user_ID__c_x,nanoHUB_user_ID__c_y,Contact ID,Venue__c_y,ven_joined
0,hk462@cam.ac.uk,NaN,195805.0,195805,0035w000034Jz2P,Long-term Users_not recently active,Long-term Users_not recently active
1,pavanello@fei.edu.br,NaN,201273.0,201273,0035w000034K3AJ,Long-term Users_not recently active,Long-term Users_not recently active
2,ahaldar@bu.edu,NaN,201547.0,201547,0035w000034K3Mz,Long-term Users_not recently active,Long-term Users_not recently active
3,284461042@qq.com,NaN,201817.0,201817,0035w000034K3ZL,Long-term Users_not recently active,Long-term Users_not recently active
4,reshmaparthasarathy06@gmail.com,NaN,209664.0,209664,0035w000034K2DT,Long-term Users_not recently active,Long-term Users_not recently active


In [33]:
from copy import deepcopy

In [34]:
tt_venues = {}

In [35]:
## join multiyear UREs
for i in tt_venue_emails.keys():
    build_temp = [] 
    
    for j,k in enumerate(tt_venue_emails[i][0]):
        build_temp += ven_joined[j].split(';')
        #build_temp.append(ven_joined[j])
    
    #ven_joined
    tt_venues[i] = deepcopy(build_temp)

In [36]:
## replace venue__c_y with the new values, filter by email
for i in tt_venues.keys():
    
    for j,k in enumerate(tt_venue_emails[i][0]):
        idf_match.iloc[k,-2] = ';'.join(tt_venues[i])

In [37]:
idf_match = idf_match.drop_duplicates(subset=['Email'])

In [38]:
# combine venue__c_x and venue__c_y
ven_x = idf_match['Venue__c_x'].to_list()
ven_y = idf_match['Venue__c_y'].to_list()
ven_joined = []

for i,j in enumerate(ven_x):
    temp = []
    if type(ven_y[i]) != float and type(ven_y[i]) != type(None):
        temp.append(ven_y[i])
    
    if type(j) != float and type(j) != type(None):
        temp.append(j)
    #print(temp)
    ven_joined.append(';'.join(temp))

In [39]:
idf_match['Venue__c'] = ven_joined

In [40]:
idf_match = idf_match.drop(columns=['Venue__c_x','Venue__c_y','ven_joined']) #'Participation year',
display(idf_match)

,Email,nanoHUB_user_ID__c_x,nanoHUB_user_ID__c_y,Contact ID,Venue__c
0,hk462@cam.ac.uk,195805.0,195805,0035w000034Jz2P,Long-term Users_not recently active
1,pavanello@fei.edu.br,201273.0,201273,0035w000034K3AJ,Long-term Users_not recently active
2,ahaldar@bu.edu,201547.0,201547,0035w000034K3Mz,Long-term Users_not recently active
3,284461042@qq.com,201817.0,201817,0035w000034K3ZL,Long-term Users_not recently active
4,reshmaparthasarathy06@gmail.com,209664.0,209664,0035w000034K2DT,Long-term Users_not recently active
...,...,...,...,...,...
11871,i.zadorozhnyi@fz-juelich.de,116789.0,116789,0035w000034JrT0,Long-term Users_not recently active
11872,nirmalharsh53@gmail.com,117582.0,117582,0035w000034JsIZ,Long-term Users_not recently active
11873,greenlight2005@gmail.com,117584.0,117584,0035w000034JsIb,Long-term Users_not recently active
11874,plapas@physics.utexas.edu,117588.0,117588,0035w000034JsIf,Long-term Users_not recently active


In [41]:
idf_match = idf_match.drop(columns=['nanoHUB_user_ID__c_x'])
idf_match = idf_match.rename(columns={'nanoHUB_user_ID__c_y':'nanoHUB_user_ID__c'})
display(idf_match.head(5))

,Email,nanoHUB_user_ID__c,Contact ID,Venue__c
0,hk462@cam.ac.uk,195805,0035w000034Jz2P,Long-term Users_not recently active
1,pavanello@fei.edu.br,201273,0035w000034K3AJ,Long-term Users_not recently active
2,ahaldar@bu.edu,201547,0035w000034K3Mz,Long-term Users_not recently active
3,284461042@qq.com,201817,0035w000034K3ZL,Long-term Users_not recently active
4,reshmaparthasarathy06@gmail.com,209664,0035w000034K2DT,Long-term Users_not recently active


In [42]:
# select only those with valid NH ids
idf_match2 = idf_match.dropna(subset=['nanoHUB_user_ID__c'])
idf_match2 = idf_match2.rename(columns={'Alternate email':'hed__AlternateEmail__c'})
display(idf_match2)

,Email,nanoHUB_user_ID__c,Contact ID,Venue__c
0,hk462@cam.ac.uk,195805,0035w000034Jz2P,Long-term Users_not recently active
1,pavanello@fei.edu.br,201273,0035w000034K3AJ,Long-term Users_not recently active
2,ahaldar@bu.edu,201547,0035w000034K3Mz,Long-term Users_not recently active
3,284461042@qq.com,201817,0035w000034K3ZL,Long-term Users_not recently active
4,reshmaparthasarathy06@gmail.com,209664,0035w000034K2DT,Long-term Users_not recently active
...,...,...,...,...
11871,i.zadorozhnyi@fz-juelich.de,116789,0035w000034JrT0,Long-term Users_not recently active
11872,nirmalharsh53@gmail.com,117582,0035w000034JsIZ,Long-term Users_not recently active
11873,greenlight2005@gmail.com,117584,0035w000034JsIb,Long-term Users_not recently active
11874,plapas@physics.utexas.edu,117588,0035w000034JsIf,Long-term Users_not recently active


In [43]:
idf_match2 = idf_match2.drop(columns=['Contact ID'])

In [44]:
display(idf_match2)

,Email,nanoHUB_user_ID__c,Venue__c
0,hk462@cam.ac.uk,195805,Long-term Users_not recently active
1,pavanello@fei.edu.br,201273,Long-term Users_not recently active
2,ahaldar@bu.edu,201547,Long-term Users_not recently active
3,284461042@qq.com,201817,Long-term Users_not recently active
4,reshmaparthasarathy06@gmail.com,209664,Long-term Users_not recently active
...,...,...,...
11871,i.zadorozhnyi@fz-juelich.de,116789,Long-term Users_not recently active
11872,nirmalharsh53@gmail.com,117582,Long-term Users_not recently active
11873,greenlight2005@gmail.com,117584,Long-term Users_not recently active
11874,plapas@physics.utexas.edu,117588,Long-term Users_not recently active


In [45]:
#send the matching ones
db_c = DB2SalesforceAPI(sf_login_params)

# send data to SF
db_c.object_id = 'Contact'
db_c.external_id = 'nanoHUB_user_ID__c'

db_c.send_data(idf_match2)

Obtained Salesforce access token ...... True
[Success] Bulk job creation successful. Job ID = 7505w00000VKWYEAA5
[Success] CSV upload successful. Job ID = 7505w00000VKWYEAA5
[Success] Closing job successful. Job ID = 7505w00000VKWYEAA5


In [52]:
# check status
db_c.check_bulk_status()

{'id': '7505w00000VKWYEAA5',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-04-16T16:02:45.000+0000',
 'systemModstamp': '2021-04-16T16:05:17.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'nanoHUB_user_ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 11543,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 142948,
 'apiActiveProcessingTime': 129133,
 'apexProcessingTime': 901674}

In [53]:
from pprint import pprint

pprint(db_c.check_bulk_failed_results())

'"sf__Id","sf__Error",Email,Venue__c,nanoHUB_user_ID__c\n'


In [ ]:
## import in the leads

In [56]:
display(idf)

,lastname,firstname,Email,MaRDA_Survey_Link__c,Venue__c
0,Pochan,Darrin,pochan@udel.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey
1,Bilionis,Ilias,ibilion@purdue.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey
2,Titus,Michael,titus9@purdue.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey
3,Sandhage,Kenneth,sandhage@purdue.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey
4,Zhang,Peihong,pzhang3@buffalo.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey
...,...,...,...,...,...
299,Silva,Carlos,carlos.silva@gatech.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey
300,Lewis,Lisa,lblewis@albion.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey
301,Kubis,Tillmann,tkubis@purdue.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey
302,West,Jared,jgwest@purdue.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey


In [57]:
# refilter brows with grows
for i,j in enumerate(brows[:2]):
    if j in grows:
        del brows[i]
print(len(brows))

259


In [58]:
lead_ids = idf.iloc[brows,:]

In [59]:
lead_ids = lead_ids.drop_duplicates(subset=['Email'])
display(lead_ids)

,lastname,firstname,Email,MaRDA_Survey_Link__c,Venue__c
0,Pochan,Darrin,pochan@udel.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey
4,Zhang,Peihong,pzhang3@buffalo.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey
5,Stevanovic,Vladan,vstevano@mines.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey
6,Swain,Greg,swain@chemistry.msu.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey
7,Hogan,Timothy,hogant@egr.msu.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey
...,...,...,...,...,...
294,Grover,Martha,martha.grover@chbe.gatech.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey
295,Reichmanis,Elsa,elsa.reichmanis@chbe.gatech.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey
296,McEuen,Paul,plm23@cornell.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey
297,Katz,Howard,hekatz@jhu.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey


In [62]:
## import all leads from SF and try to generate new SF_indexers
#pull all current leads
db_s = DB2SalesforceAPI(sf_login_params)
sf_df = db_s.query_data('SELECT Email, Venue__c, SF_indexer__c FROM Lead')


Obtained Salesforce access token ...... True
[Success] Bulk job creation successful. Job ID = 7505w00000VJuxuAAD
{"id":"7505w00000VJuxuAAD","operation":"query","object":"Lead","createdById":"0055w00000DM5bOAAT","createdDate":"2021-04-13T14:12:48.000+0000","systemModstamp":"2021-04-13T14:12:48.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000VJuxuAAD","operation":"query","object":"Lead","createdById":"0055w00000DM5bOAAT","createdDate":"2021-04-13T14:12:48.000+0000","systemModstamp":"2021-04-13T14:12:50.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":7447,"retries":0,"totalProcessingTime":1083}
[Success] Bulk job completed successfully.


In [63]:
# find the max sf_indexer
indexers = sf_df['SF_indexer__c'].fillna(0).to_list()
max_ind = max(indexers)

In [64]:
max_ind

2946.0

In [65]:
# find all existing leads
sf_emails = sf_df['Email'].to_list()
m_rows = []
nm_rows = [] #don't need sf no match rows
sf_mrows = []

In [66]:
for ind,val in enumerate(lead_ids['Email'].to_list()):
    if val in sf_emails:
        m_rows.append(ind)
        sf_mrows.append(sf_emails.index(val))
    else:
        nm_rows.append(ind)

In [67]:
# filter the matches
sf_df_match = sf_df.iloc[sf_mrows,:].reset_index().iloc[:,1:]
join_idf = lead_ids.iloc[m_rows,:].reset_index().iloc[:,1:]
new_idf = lead_ids.iloc[nm_rows,:].reset_index().iloc[:,1:]

In [68]:
## Alternate_Email__c"
display(sf_df_match.head(2))
display(join_idf.head(2))

,Email,SF_indexer__c,Venue__c
0,wang.363@osu.edu,2759.0,nanoMFG email list
1,fso@ncsu.edu,1352.0,NaN


,lastname,firstname,Email,MaRDA_Survey_Link__c,Venue__c
0,Wang,Yunzhi,wang.363@osu.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey
1,So,Franky,fso@ncsu.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey


In [69]:
# linear join since the sequence is matching
for ind,val in enumerate(sf_df_match['Venue__c']):
    try:
        val = val.split(';')
        #if 'MSE Summer Webinar Series 2020' in val:
        #    val.remove('MSE Summer Webinar Series 2020')
        #    if 'MSE Summer Webinar Series 2020' in val:
        #        val.remove('MSE Summer Webinar Series 2020')
        val.append(join_idf['Venue__c'][ind])
        val = ';'.join(val)
    except:
        val = join_idf['Venue__c'][ind]
    sf_df_match['Venue__c'][ind] = val    

print(sf_df_match.head(5))

                Email  SF_indexer__c  \
0    wang.363@osu.edu         2759.0   
1        fso@ncsu.edu         1352.0   
2           hu@sc.edu           61.0   
3  ankit.sri@tamu.edu         1240.0   
4        liju@mit.edu         1577.0   

                                            Venue__c  
0               nanoMFG email list;MaRDA 2021 Survey  
1                                  MaRDA 2021 Survey  
2                                  MaRDA 2021 Survey  
3  MSE Delivra Engagement;MSE SWS 2020 - S;MaRDA ...  
4  MSE Delivra Engagement;MSE SWS 2020 - S;MaRDA ...  


C:\Users\henry\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [70]:
## delete duplicates
venues = sf_df_match['Venue__c'].to_list()
for ind,val in enumerate(venues):
    venues[ind] = ';'.join(list(dict.fromkeys(val.split(';'))))
sf_df_match['Venue__c'] = venues
display(sf_df_match.head(5))

,Email,SF_indexer__c,Venue__c
0,wang.363@osu.edu,2759.0,nanoMFG email list;MaRDA 2021 Survey
1,fso@ncsu.edu,1352.0,MaRDA 2021 Survey
2,hu@sc.edu,61.0,MaRDA 2021 Survey
3,ankit.sri@tamu.edu,1240.0,MSE Delivra Engagement;MSE SWS 2020 - S;MaRDA ...
4,liju@mit.edu,1577.0,MSE Delivra Engagement;MSE SWS 2020 - S;MaRDA ...


In [71]:
# assign new SF_indexers for the new leads
new_max_ind = int(max_ind+new_idf.shape[0])
# print(new_idf.shape)
# print(max_ind)
# print(new_max_ind)
new_idf['SF_indexer__c'] = range(int(max_ind)+1,new_max_ind+1)

In [72]:
# need non-empty company field
new_idf['Company'] = '-'

In [73]:
new_idf.head(5)

,lastname,firstname,Email,MaRDA_Survey_Link__c,Venue__c,SF_indexer__c,Company
0,Pochan,Darrin,pochan@udel.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey,2947,-
1,Zhang,Peihong,pzhang3@buffalo.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey,2948,-
2,Stevanovic,Vladan,vstevano@mines.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey,2949,-
3,Swain,Greg,swain@chemistry.msu.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey,2950,-
4,Hogan,Timothy,hogant@egr.msu.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey,2951,-


In [74]:
# ensure 'â\x80\x93' has been replaced
sf_df_match['Venue__c'] = sf_df_match['Venue__c'].apply(lambda x: x.replace('â\x80\x93','-'))
new_idf['Venue__c'] = new_idf['Venue__c'].apply(lambda x: x.replace('â\x80\x93','-'))

display(sf_df_match.head(5))
display(new_idf.head(5))

,Email,SF_indexer__c,Venue__c
0,wang.363@osu.edu,2759.0,nanoMFG email list;MaRDA 2021 Survey
1,fso@ncsu.edu,1352.0,MaRDA 2021 Survey
2,hu@sc.edu,61.0,MaRDA 2021 Survey
3,ankit.sri@tamu.edu,1240.0,MSE Delivra Engagement;MSE SWS 2020 - S;MaRDA ...
4,liju@mit.edu,1577.0,MSE Delivra Engagement;MSE SWS 2020 - S;MaRDA ...


,lastname,firstname,Email,MaRDA_Survey_Link__c,Venue__c,SF_indexer__c,Company
0,Pochan,Darrin,pochan@udel.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey,2947,-
1,Zhang,Peihong,pzhang3@buffalo.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey,2948,-
2,Stevanovic,Vladan,vstevano@mines.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey,2949,-
3,Swain,Greg,swain@chemistry.msu.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey,2950,-
4,Hogan,Timothy,hogant@egr.msu.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey,2951,-


In [75]:
sf_df_match['MaRDA_Survey_Link__c'] = '-'
marda_list = []
survey_links = lead_ids['MaRDA_Survey_Link__c'].to_list()
lead_email_list = lead_ids['Email'].to_list()

for i,j in enumerate(sf_df_match['Email'].to_list()):
    ind_val = lead_email_list.index(j)
    marda_list.append(survey_links[ind_val])


In [76]:
sf_df_match['MaRDA_Survey_Link__c'] = marda_list
display(sf_df_match.head(5))

,Email,SF_indexer__c,Venue__c,MaRDA_Survey_Link__c
0,wang.363@osu.edu,2759.0,nanoMFG email list;MaRDA 2021 Survey,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...
1,fso@ncsu.edu,1352.0,MaRDA 2021 Survey,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...
2,hu@sc.edu,61.0,MaRDA 2021 Survey,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...
3,ankit.sri@tamu.edu,1240.0,MSE Delivra Engagement;MSE SWS 2020 - S;MaRDA ...,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...
4,liju@mit.edu,1577.0,MSE Delivra Engagement;MSE SWS 2020 - S;MaRDA ...,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...


In [82]:
## add companies

In [83]:
## check intactness of idf
display(idf.head(2))
print(idf.shape)
idf.columns

,lastname,firstname,Email,MaRDA_Survey_Link__c,Venue__c
0,Pochan,Darrin,pochan@udel.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey
1,Bilionis,Ilias,ibilion@purdue.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey


(304, 5)


Index(['lastname', 'firstname', 'Email', 'MaRDA_Survey_Link__c', 'Venue__c'], dtype='object')

In [84]:
from copy import deepcopy
sf_df_match['Company'] = '-'
new_idf['Company'] = '-'

In [85]:
# populate the company fields for sf_df_match and new_idf by comparing email addresses
if 'Company' in idf.columns:
    print(True)
    # comparison for sf_df_match

    sf_df_match_comp_ind = sf_df_match.columns.to_list().index('Company')
    for ind,val in enumerate(idf['Email'].to_list()):
        if val in sf_df_match['Email'].to_list():
            sf_df_match_ind = sf_df_match['Email'].to_list().index(val)
            sf_df_match.iloc[sf_df_match_ind,sf_df_match_comp_ind] = deepcopy(idf['Company'].to_list()[ind])

    # sf_df_match sf_indexer if not available
    sf_df_match_sf_ind = sf_df_match.columns.to_list().index('SF_indexer__c')
    new_max_ind = int(max_ind+new_idf.shape[0])
    for ind,val in enumerate(sf_df_match['SF_indexer__c'].to_list()):
        if type(val) != int and type(val) != float:
            sf_df_match.iloc[ind,sf_df_match_sf_ind] = new_max_ind+1
            new_max_ind += 1
    
    new_idf_comp_ind = new_idf.columns.to_list().index('Company')
    for ind,val in enumerate(idf['Email'].to_list()):
        if val in new_idf['Email'].to_list():
            new_idf_ind = new_idf['Email'].to_list().index(val)
            new_idf.iloc[new_idf_ind,new_idf_comp_ind] = deepcopy(idf['Company'].to_list()[ind])
else:
    print('no company in import list, set to -')

no company in import list, set to -


In [86]:
sf_df_match = sf_df_match.fillna('-')
new_idf = new_idf.fillna('-')
display(sf_df_match.head(5))
display(new_idf.head(5))

,Email,SF_indexer__c,Venue__c,MaRDA_Survey_Link__c,Company
0,wang.363@osu.edu,2759,nanoMFG email list;MaRDA 2021 Survey,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,-
1,fso@ncsu.edu,1352,MaRDA 2021 Survey,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,-
2,hu@sc.edu,61,MaRDA 2021 Survey,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,-
3,ankit.sri@tamu.edu,1240,MSE Delivra Engagement;MSE SWS 2020 - S;MaRDA ...,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,-
4,liju@mit.edu,1577,MSE Delivra Engagement;MSE SWS 2020 - S;MaRDA ...,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,-


,lastname,firstname,Email,MaRDA_Survey_Link__c,Venue__c,SF_indexer__c,Company
0,Pochan,Darrin,pochan@udel.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey,2947,-
1,Zhang,Peihong,pzhang3@buffalo.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey,2948,-
2,Stevanovic,Vladan,vstevano@mines.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey,2949,-
3,Swain,Greg,swain@chemistry.msu.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey,2950,-
4,Hogan,Timothy,hogant@egr.msu.edu,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,MaRDA 2021 Survey,2951,-


In [87]:
#new_idf = new_idf.drop(columns=['Campaign','Business Phone 2','Business Fax',"Assistant's Phone",'Job Title'])
#new_idf = new_idf.drop(columns=['Campaign','Job Title'])
new_idf = new_idf[['Email','firstname','lastname','SF_indexer__c','Venue__c','MaRDA_Survey_Link__c']]
new_idf['Company'] = '-'

In [88]:
display(new_idf.head(2))

,Email,firstname,lastname,SF_indexer__c,Venue__c,MaRDA_Survey_Link__c,Company
0,pochan@udel.edu,Darrin,Pochan,2947,MaRDA 2021 Survey,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,-
1,pzhang3@buffalo.edu,Peihong,Zhang,2948,MaRDA 2021 Survey,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,-


In [89]:
#drop duplicate rows
sf_df_match = sf_df_match.drop_duplicates(subset='SF_indexer__c')
new_idf = new_idf.drop_duplicates()

In [97]:
sf_indexers = []

for i,j in enumerate(sf_df_match['SF_indexer__c'].to_list()):
    if j == '-':
        sf_indexers.append(int(3500))
    else:
        sf_indexers.append(int(j))
print(sf_indexers)

[2759, 1352, 61, 1240, 1577, 1540, 952, 2682, 957, 1399, 1799, 854, 1179, 1406, 74, 1408, 1137, 1168, 1290, 1545, 1572, 1938, 1570, 1376, 1176, 1223, 689, 1186, 1366, 3500, 1546, 959, 1385, 875]


In [98]:
sf_df_match['SF_indexer__c'] = sf_indexers

In [105]:
display(sf_df_match.tail(5))
print(sf_df_match.shape)

,Email,SF_indexer__c,Venue__c,MaRDA_Survey_Link__c,Company
29,sghosh20@jhu.edu,3500,MaRDA 2021 Survey,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,-
30,seshadri@mrl.ucsb.edu,1546,MSE Delivra Engagement;MSE SWS 2020 - S;MaRDA ...,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,-
31,jihlefeld@virginia.edu,959,MaRDA 2021 Survey,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,-
32,bazan@chem.ucsb.edu,1385,MSE Delivra Engagement;MSE SWS 2020 - S;MaRDA ...,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,-
33,mgrayson@eecs.northwestern.edu,875,MaRDA 2021 Survey,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,-


(34, 5)


In [107]:
sf_df_match.iloc[29,3]

'https://purdue.ca1.qualtrics.com/jfe/form/SV_0e7rmMFvplVnr9P?Q_DL=sPBwGhwoCHLK4r2_0e7rmMFvplVnr9P_MLRP_1Rg5zb8ziXS6Omq&Q_CHL=gl'

In [100]:
display(new_idf)

,Email,firstname,lastname,SF_indexer__c,Venue__c,MaRDA_Survey_Link__c,Company
0,pochan@udel.edu,Darrin,Pochan,2947,MaRDA 2021 Survey,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,-
1,pzhang3@buffalo.edu,Peihong,Zhang,2948,MaRDA 2021 Survey,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,-
2,vstevano@mines.edu,Vladan,Stevanovic,2949,MaRDA 2021 Survey,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,-
3,swain@chemistry.msu.edu,Greg,Swain,2950,MaRDA 2021 Survey,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,-
4,hogant@egr.msu.edu,Timothy,Hogan,2951,MaRDA 2021 Survey,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,-
...,...,...,...,...,...,...,...
219,carson.meredith@chbe.gatech.edu,Carson,Meredith,3166,MaRDA 2021 Survey,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,-
220,martha.grover@chbe.gatech.edu,Martha,Grover,3167,MaRDA 2021 Survey,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,-
221,plm23@cornell.edu,Paul,McEuen,3168,MaRDA 2021 Survey,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,-
222,hekatz@jhu.edu,Howard,Katz,3169,MaRDA 2021 Survey,https://purdue.ca1.qualtrics.com/jfe/form/SV_0...,-


In [102]:
#send the matching ones
db_s_l1 = DB2SalesforceAPI(sf_login_params)

# send data to SF
db_s_l1.object_id = 'Lead'
db_s_l1.external_id = 'SF_indexer__c'

db_s_l1.send_data(sf_df_match)

Obtained Salesforce access token ...... True
[Success] Bulk job creation successful. Job ID = 7505w00000VJv4bAAD
[Success] CSV upload successful. Job ID = 7505w00000VJv4bAAD
[Success] Closing job successful. Job ID = 7505w00000VJv4bAAD


In [103]:
# check status
db_s_l1.check_bulk_status()

{'id': '7505w00000VJv4bAAD',
 'operation': 'upsert',
 'object': 'Lead',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-04-13T14:25:28.000+0000',
 'systemModstamp': '2021-04-13T14:25:30.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'SF_indexer__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 34,
 'numberRecordsFailed': 1,
 'retries': 0,
 'totalProcessingTime': 383,
 'apiActiveProcessingTime': 284,
 'apexProcessingTime': 49}

In [104]:
from pprint import pprint

pprint(db_s_l1.check_bulk_failed_results())

('"sf__Id","sf__Error",Company,Email,MaRDA_Survey_Link__c,SF_indexer__c,Venue__c\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","-","sghosh20@jhu.edu","https://purdue.ca1.qualtrics.com/jfe/form/SV_0e7rmMFvplVnr9P?Q_DL=sPBwGhwoCHLK4r2_0e7rmMFvplVnr9P_MLRP_1Rg5zb8ziXS6Omq&Q_CHL=gl","3500.0","MaRDA '
 '2021 Survey"\n')


In [108]:
#send the new ones
db_s_l2 = DB2SalesforceAPI(sf_login_params)

# send data to SF
db_s_l2.object_id = 'Lead'
db_s_l2.external_id = 'SF_indexer__c'

db_s_l2.send_data(new_idf)

Obtained Salesforce access token ...... True
[Success] Bulk job creation successful. Job ID = 7505w00000VJv6IAAT
[Success] CSV upload successful. Job ID = 7505w00000VJv6IAAT
[Success] Closing job successful. Job ID = 7505w00000VJv6IAAT


In [112]:
# check status
db_s_l2.check_bulk_status()

{'id': '7505w00000VJv6IAAT',
 'operation': 'upsert',
 'object': 'Lead',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-04-13T14:28:40.000+0000',
 'systemModstamp': '2021-04-13T14:28:52.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'SF_indexer__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 224,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 1866,
 'apiActiveProcessingTime': 1474,
 'apexProcessingTime': 110}

In [113]:
from pprint import pprint

pprint(db_s_l2.check_bulk_failed_results())

'"sf__Id","sf__Error",Company,Email,MaRDA_Survey_Link__c,SF_indexer__c,Venue__c,firstname,lastname\n'
